In [116]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [117]:
# Per prima cosa importiamo la libreria trasformers e il dataset di interesse

!pip install transformers

datasim = pd.read_csv('../input/dataset-di-poesie-italiane-800900/Dataset similitudini.txt')
datasim

In [118]:
#essendo il dataset originale ordinato per label, ci assicuriamo di randomizzarlo prima di utilizzarlo

dfsim = datasim.sample(frac=1).reset_index(drop=True)
dfsim

In [119]:
#Suddividiamo il dataset in train set e test set dopo aver randomizzato l'ordine dei record.

import math
n_train = math.floor(180*0.8)

train_df = dfsim[:n_train]
test_df = dfsim[n_train:]


# Mostriamo il numero di frasi per le due suddivisioni
print('Numero di frasi di training: {:,}\n'.format(train_df.shape[0]))
print('Numero di frasi di testing: {:,}\n'.format(test_df.shape[0]))

In [120]:
# Visualizziamo 10 frasi random dal dataset di training creato.

train_df.sample(10)

Per poter essere utilizzate come input nel modello BERT, le frasi e le corrispondenti etichette devono essere prima **preprocessate**.
Prima si estraggono i valori (testo e label) separatamente, dopo si procede con la **tokenizzazione** del testo e la **mappatura dei tokens** sul corrispettivo indice del vocabolario.

In [121]:
#essendo l'input salvato in due colonne, si devono prima concatenare i due campi

train_labels = train_df.label_sim.values
trs1 = [i if type(i)==str else "" for i in list(train_df.termine_uno.values)]
trs2 = list(train_df.termine_due.values)
train_sentences = []
for a,b in zip(trs1,trs2):
    train_sentences.append(a+" "+b)
train_sentences = np.array(train_sentences)

test_labels = test_df.label_sim.values
tes1 = [i if type(i)==str else "" for i in list(test_df.termine_uno.values)]
tes2 = list(test_df.termine_due.values)
test_sentences = []
for a,b in zip(tes1,tes2):
    test_sentences.append(a+" "+b)
test_sentences = np.array(test_sentences)

In [122]:
# Utilizziamo un tokenizer preallenato sulla lingua italiana

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-cased")

In [123]:
# Visualizziamo il risultato del tokenizzatore su un'altra frase

print(' Originale: ', train_sentences[10])

print('Tokenizzata: ', tokenizer.tokenize(train_sentences[10]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sentences[10])))

Per formattare correttamente l'input del modello BERT dovremo:

1. aggiungere dei token speciali all'inizio e alla fine di ogni frase (**'[CLS]'** e **'[SEP]'**),
2. aggiungere **padding** e **troncare** le frasi in modo che i tensori corrispondenti abbiano tutti la stessa lunghezza,
3. differenziare i token "pieni" dai token di padding attraverso il tensore di **attention mask**.

Per stabilire la lunghezza finale ideale delle frasi si è calcolata prima la lunghezza e poi la distribuzione delle lunghezze all'interno del corpus.

In [124]:
# Calcoliamo la stringa di testo più lungo nel dataset di training

max_len_train = 0
for sent in train_sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len_train = max(max_len_train, len(input_ids))

print('Lunghezza massima: ', max_len_train)

In [125]:
# Visualizziamo la distribuzione delle lunghezze 

from matplotlib import pyplot as plt
import seaborn as sns

tokens_len = []

for sent in train_sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    tokens_len.append(len(input_ids))
    
sns.distplot(tokens_len)

In [126]:
# Visualizzando i dati in boxplot si evidenziano gli outliers

sns.boxplot(tokens_len)

Procediamo con la **tokenizzazione**, **definizione degli ID** e delle **attention mask** per tutte le frasi del dataset di training. 
Dato che la lunghezza massima è comunque piccola rispetto a quella supportata, si è scelto di usare un valore più alto per assicurarci che non avvengano troncamenti di alcun tipo.

In [127]:
input_ids = []
attention_masks = []

for sent in train_sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Aggiunta di '[CLS]' e '[SEP]'
                        max_length = 80,          # Padding
                        padding= 'max_length',
                        truncation=True,
                        return_attention_mask = True,   # Costruzione delle att. mask.
                        return_tensors = 'pt',     # Restituzione dei tensori pytorch
                   )
        
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

# Trasformiamo le liste in tensori
train_input_ids = torch.cat(input_ids, dim=0)
train_attention_masks = torch.cat(attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# Visualizziamo il risultato per una frase
print('Originale: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])
print('Attention mask', train_attention_masks[0])

Seguiamo lo stesso procedimento per il dataset di test.

In [128]:
max_len = 0

for sent in test_sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    
tokens_len = []

for sent in test_sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    tokens_len.append(len(input_ids))

print('Lunghezza massima: ', max_len)
sns.distplot(tokens_len)

In [129]:
sns.boxplot(tokens_len)

In [130]:
input_ids = []
attention_masks = []


for sent in test_sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True,
                        max_length = 80,           
                        padding='max_length',
                        truncation=True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
      
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(input_ids, dim=0)
test_attention_masks = torch.cat(attention_masks, dim=0)
test_labels = torch.tensor(test_labels)

print('Original: ', test_sentences[10])
print('Token IDs:', test_input_ids[10])
print('Attention mask', test_attention_masks[10])

Il dataset viene diviso randomicamente in 90% training set e 10% validation set

In [131]:
from torch.utils.data import TensorDataset, random_split

# Si combinano gli input di training in un TensorDataset.
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Calcoliamo il numero di samples da includere in ciascun set.
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size

# Dividiamo il dataset randomicamente.
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

Dato che il task di nostro interesse consiste nella classificazione di frasi, importiamo la classe **BertForSequenceClassification** per il fine-tuning del modello BERT.

In [132]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
 
model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-italian-xxl-cased", # lo stesso usato per il tokenizzatore
    num_labels = 2,  # il numero di classi che ci interessa
    output_attentions = False, 
    output_hidden_states = False,
)

In [133]:
from torch.utils.data import DataLoader
from torch.nn import functional as F


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model.to(device)
model.train()

optim = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

loaders = {"train" : train_loader,
           "val" : val_loader,
           "test" : test_loader}

history_loss = {"train":[], "val":[], "test":[]}
history_accuracy = {"train":[], "val":[], "test":[]}

try:
  for epoch in range(5):
    sum_loss = {"train":0, "val":0, "test":0}
    sum_accuracy = {"train":0, "val":0, "test":0}
    for split in ["train","val","test"]:
      print(f"Processing {split}")
      for batch in loaders[split]:
          optim.zero_grad()
          # `batch` contains three pytorch tensors:
          #  [0]: input ids 
          #  [1]: attention masks
          #  [2]: labels
          input_ids = batch[0].to(device)
          attn_mask = batch[1].to(device)
          labels = batch[2].to(device)
          output = model(input_ids, token_type_ids=None, attention_mask=attn_mask, labels=labels, return_dict=True)
          loss = output[0]
          #logits -> batch_size x num_labels
          logits = output[1]
          sum_loss[split] += loss.item()
          if split == "train":
            loss.backward()
            optim.step()
          pred = torch.argmax(logits, dim = 1)
          batch_accuracy = (pred == labels).sum().item()/(input_ids.size(0))
          sum_accuracy[split] += batch_accuracy
    epoch_loss = {split: sum_loss[split]/len(loaders[split]) for split in ["train","val","test"]}
    epoch_accuracy = {split: sum_accuracy[split]/len(loaders[split]) for split in ["train","val","test"]}
    for split in["train","val","test"]:
      history_loss[split].append(epoch_loss[split])
      history_accuracy[split].append(epoch_accuracy[split])
    print(f"Epoch {epoch+1}:",
          f"TrL={epoch_loss['train']:.4f},",
          f"TrA={epoch_accuracy['train']:.4f},",
          f"VL={epoch_loss['val']:.4f},",
          f"VA={epoch_accuracy['val']:.4f},",
          f"TeL={epoch_loss['test']:.4f},",
          f"TeA={epoch_accuracy['test']:.4f},")
except KeyboardInterrupt:
  print("interrupted")
finally:
  plt.title("loss")
  for split in ["train","val","test"]:
    plt.plot(history_loss[split], label=split)
  plt.legend()
  plt.show()

  plt.title("accuracy")
  for split in ["train","val","test"]:
    plt.plot(history_accuracy[split], label=split)
  plt.legend()
  plt.show()
    
#torch.save(model.state_dict(), "./modello5.4")

In [134]:
model.eval()

Per avere ulteriori metriche di riferimento per la valutazione del modello, abbiamo deciso di plottare la **matrice di confusione** e di calcolare il **classification report** sui risultati della parte di test.

In [135]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

x_test = test_sentences
y_test = test_labels

y_pred=[]
for x in x_test:
    input_encodings = tokenizer(x, return_tensors='pt')
    input_ids = input_encodings["input_ids"].to(device)
    attn_mask = input_encodings["attention_mask"].to(device)
    output = model(input_ids, attention_mask=attn_mask)
    pred = torch.argmax(output[0], dim = 1)
    y_pred.append(pred.item())    
    
print(classification_report(y_test,y_pred))

In [136]:
cm= confusion_matrix(y_test,y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot() 

Possiamo testare manualmente il nostro modello con frasi di prova.

In [137]:
frase = "voglio che sia freddo come il ghiaccio"

input_encodings = tokenizer(frase, return_tensors='pt')
input_ids = input_encodings["input_ids"].to(device)
attn_mask = input_encodings["attention_mask"].to(device)
output = model(input_ids, attention_mask=attn_mask)
pred = torch.argmax(output[0], dim = 1)
print(pred.item())

In [138]:
frase = "m'illumino d'immenso"

input_encodings = tokenizer(frase, return_tensors='pt')
input_ids = input_encodings["input_ids"].to(device)
attn_mask = input_encodings["attention_mask"].to(device)
output = model(input_ids, attention_mask=attn_mask)
pred = torch.argmax(output[0], dim = 1)
print(pred.item())